In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [2]:
df = pd.read_csv(r'lie_detection_dataset.csv')

In [3]:
df.head()

,gsr_value,flex_value,gaze_aversion,pupil_dilation,blink_rate,lie_probability
0,87.42,5.52,0.93,8.12,0.555,0.435
1,191.13,10.31,1.53,5.40,0.115,0.849
2,151.76,10.41,0.97,7.72,0.113,0.783
3,127.76,11.37,0.49,13.75,0.294,0.807
4,48.08,12.26,0.66,11.32,0.393,0.587


In [4]:
y = df['lie_probability']
x = df.drop(columns = ['lie_probability'])

In [5]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size = 0.2, random_state = 42)

In [7]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  
])

C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

In [9]:
model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - loss: 0.2823 - mae: 0.4349 - val_loss: 0.1768 - val_mae: 0.3165
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.1211 - mae: 0.2746 - val_loss: 0.0965 - val_mae: 0.2407
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0782 - mae: 0.2178 - val_loss: 0.0563 - val_mae: 0.1984
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0430 - mae: 0.1756 - val_loss: 0.0319 - val_mae: 0.1554
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0317 - mae: 0.1443 - val_loss: 0.0192 - val_mae: 0.1162
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0239 - mae: 0.1220 - val_loss: 0.0172 - val_mae: 0.1022
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0249 - mae: 0.1263 - val_loss: 0.0174 - val_mae: 0.1016
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0232 - mae: 0.1198 - val_loss: 0.0176 - val_mae: 0.1034
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0201 - mae: 

In [10]:
loss, mae = model.evaluate(x_test, y_test)
print(f"Test MAE: {mae:.4f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0245 - mae: 0.1108
Test MAE: 0.1095


In [17]:
input_data = np.array([[400, 8, 2.32, 18.747, 0.0195]])
prediction = model.predict(input_data)
print("Predicted Output:", prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Predicted Output: [[81.011795]]


In [20]:
model.export("project_model") 

INFO:tensorflow:Assets written to: project_model\assets


INFO:tensorflow:Assets written to: project_model\assets


Saved artifact at 'project_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 5), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  1756163571408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1756163892576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1756163896096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1756163896624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1756163898032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1756163896272: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [22]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model("project_model")
tflite_model = converter.convert()

In [23]:
with open("model.tflite", "wb") as f:
    f.write(tflite_model)